In [1]:
from sub_func import *
from pf_selection import *
from pipeline_utils import *

from t1 import *
from t2_t4 import *
from t5 import *

/opt/anaconda3/envs/finTF/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


예시로, 2023년 1분기 분석을 위해 2022년 4분기 정보를 바탕으로 2023년 1분기 초기 포트폴리오 생성

In [2]:
# 조회할 정보의 기간
base_year = '2022'
base_quarter = 'Q4'
start_date = '20221001'
end_date = '20221231'

# 포트폴리오를 활용하는 실제 기간 (ex. 2022 Q4를 조회하여 2023 Q1을 위한 포트폴리오 생성)
target_year = '2023'
target_quarter = 'Q1'

# 'pf_selection_agent'에 초기 포트폴리오가 있는지 확인
db_info = get_all_page_ids_from_database('pf_selection_agent')
init_pf_page_title = f"{target_year}_{target_quarter}_init_pf"

if init_pf_page_title not in list(db_info.keys()):
    print('초기 포트폴리오를 생성합니다...')
    pf_selection_info = pf_selection_main(base_year, base_quarter, start_date, end_date)
    print("노션에 output을 모두 저장합니다...")
    to_DB('pf_selection_agent', init_pf_page_title, f"{start_date}_{end_date}", str(pf_selection_info))

# 'pf_selection_agent'에서 종목 코드 가져오기
temp_tickers = get_tickers_from_json_v2('pf_selection_agent', f'{target_year}_{target_quarter}_init_pf')
tickers = []

for ticker in temp_tickers:
    if not is_delisted(ticker, today):
        tickers.append(ticker)
        
weight_dict = {ticker: 1 / len(tickers) for ticker in tickers}
 
# JSON 파일 저장할 디렉토리 경로
log_dir = f"./pf_logs/{target_year}_{target_quarter}"
os.makedirs(log_dir, exist_ok=True)  # 디렉토리 없으면 생성

# 파일 경로
file_path = os.path.join(log_dir, f"{target_year}_{target_quarter}_init_pf.json")

with open(f'{file_path}', "w", encoding="utf-8") as f:
    json.dump(weight_dict, f, indent=2, ensure_ascii=False)

In [3]:
# 오늘 날짜 (코스피 장이 열린 날만 한정, 예시)
kospi_open_days = get_kospi_open_days("20230101", "20230331")

In [4]:
def finTF(today, tickers, base_year, base_quarter, target_year, target_quarter, start_date):

    print('***** t1_analyst_main 실행 *****')
    t1_analyst_main(today, tickers, base_year, base_quarter)

    print('***** t1_pf_manager_main 실행 *****')
    t1_pf_manager_main(today, tickers, base_year, base_quarter, target_year, target_quarter)

    print('***** t1_trader_main 실행 *****')
    t1_trader_main(today, tickers, base_year, base_quarter)

    print('***** t2_t4_main 실행 *****')
    t_2_t_4_main(tickers, start_date, today, target_year, target_quarter, base_year, base_quarter)

    print('***** t5_main 실행 *****')
    t_5_main(today, base_year, base_quarter)

In [9]:
temp_tickers = get_tickers_from_json_v2('pf_selection_agent', f'{target_year}_{target_quarter}_init_pf')
tickers = []

for ticker in temp_tickers:
    if not is_delisted(ticker, today):
        tickers.append(ticker)

tickers

['020000', '031430', '093050', '004370', '000210', '282330', '111770']

In [15]:
kospi_open_days[55]

'20230323'

In [16]:
test_span = kospi_open_days[54:]
# 0227부터 재개

for today in test_span:
    finTF(today, tickers, base_year, base_quarter, target_year, target_quarter, start_date)
    print(f'{today}일자 실행 완료')

***** t1_analyst_main 실행 *****
[t1_analyst] === 종목별 분석 시작 ===
UK에 대해 Inflation Rate 정보를 찾을 수 없습니다. | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/get_info/../../../store_data/raw/FRED/UK/Inflation Rate/2022/2022_Inflation Rate.csv'

=== 020000 분석 중... ===
재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/get_info/../../../store_data/raw/opendart/store_financial_statement/020000/_020000_재무제표 ().csv'
020000의 이전 년도 재무 데이터를 불러올 수 없습니다.
020000의 fin_statement_info 정보를 확인할 수 없습니다.
analyze_financial_data의 GPT 요청 결과: {'id': 'chatcmpl-AxwiAE8Vvnzt9yaTmKKNjkvxCWlqa', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': '# 재무제표 요약 및 분석\n\n## 1. 재무상태표 분석\n\n### 자산 분석\n- **유동자산**: 1,031,083,371,680 원 (36기) / 881,759,156,232 원 (35기)\n  - 유동자산이 전년 대비 17% 증가하여 유동성이 개선됨.\n- **비유동자산**: 653,632,126,914 원 (36기) / 675,325,601,664 원 (35기)\n  - 비유동자산은 소폭 감소하였으나, 전

/Users/yeonsuk/investment/finTF/pipeline/t1.py:470: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corp_news_df['SA_result'] = corp_news_df['news_title'].apply(lambda x:



=== 031430 분석 중... ===
analyze_financial_data의 GPT 요청 결과: {'id': 'chatcmpl-AxwjCENZvKOlzZNmtkvlwRQrhwaiV', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': '# 재무 분석 보고서\n\n## 1. 요약재무정보\n### 1.1 연결재무상태표\n- **유동자산**: 547,064,855,331 원 (2022년) / 459,823,817,817 원 (2021년) / 426,786,183,462 원 (2020년)\n- **비유동자산**: 731,616,959,303 원 (2022년) / 699,303,760,568 원 (2021년) / 720,668,262,757 원 (2020년)\n- **자산총계**: 1,278,681,814,634 원 (2022년) / 1,159,127,578,385 원 (2021년) / 1,147,454,446,219 원 (2020년)\n\n### 1.2 손익계산서\n- **매출액**: 1,553,877,316,519 원 (2022년) / 1,450,778,572,821 원 (2021년) / 1,325,499,683,994 원 (2020년)\n- **영업이익**: 115,270,511,523 원 (2022년) / 91,971,396,487 원 (2021년) / 33,764,331,574 원 (2020년)\n- **당기순이익**: 118,792,238,428 원 (2022년) / 82,629,049,731 원 (2021년) / 50,958,180,102 원 (2020년)\n\n## 2. 주요 재무 비율\n- **주가**: 24,750 원\n- **PER**: 7.44\n- **PBR**: 1.06\n- **ROE**: 14.25%\n- **이익 성장률**: 43.77%\n- **CAGR**: 43.77%\n\n## 3. 분석 및 의견\n1. **자산 

/Users/yeonsuk/investment/finTF/pipeline/t1.py:470: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corp_news_df['SA_result'] = corp_news_df['news_title'].apply(lambda x:



=== 093050 분석 중... ===
analyze_financial_data의 GPT 요청 결과: {'id': 'chatcmpl-Axwk2qCePQbDJJB2ROSnlWERX9wVJ', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': '# 재무제표 분석 및 투자 의견\n\n## 1. 재무제표 요약\n\n### 1.1. 연결 재무상태표 (2022년 12월 말)\n\n- **자산총계**: 2,584,040 백만원\n- **부채총계**: 927,192 백만원\n- **자본총계**: 1,656,848 백만원\n- **비유동자산**: 1,238,270 백만원 (전년 대비 증가)\n- **유동자산**: 1,345,770 백만원 (전년 대비 증가)\n\n### 1.2. 연결 손익계산서 (2022년)\n\n- **매출액**: 1,968,540 백만원 (전년 대비 증가)\n- **영업이익**: 185,161 백만원 (전년 대비 증가)\n- **당기순이익**: 177,303 백만원 (전년 대비 증가)\n\n### 1.3. 주요 재무 비율\n\n- **주가 (Stock Price)**: 15,900 원\n- **PER (주가수익비율)**: 2.62\n- **PBR (주가순자산비율)**: 0.28\n- **ROE (자기자본이익률)**: 10.70%\n- **Profit Growth Rate**: 30.22%\n- **CAGR**: 30.22%\n\n## 2. 재무 분석\n\n### 2.1. 자산 및 자본 구조\n\n- 비유동자산과 유동자산 모두 전년 대비 증가했으며, 이는 회사가 지속적으로 성장하고 있다는 신호로 해석될 수 있습니다.\n- 자산 총계가 증가하면서 자본이 증가하였고, 이는 안정적인 재무 구조를 보여줍니다.\n\n### 2.2. 수익성\n\n- 매출액, 영업이익, 당기순이익 모두 전년 대비 증가하였으며, 특히 매출액의 성장률이 30%를 초과한 것은 

...

=== 6/7 ===


python(8194) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(8195) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(8196) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(8197) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(8198) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(8199) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(8201) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(8202) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(8203) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(8204) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(8205) MallocStackLoggin

=== 7/7 ===


python(8459) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(8460) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(8461) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(8462) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(8463) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(8464) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(8465) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(8466) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(8467) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(8468) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(8469) MallocStackLoggin

=== [9] 1/7 (020000) ===
=== [9] 2/7 (031430) ===
=== [9] 3/7 (093050) ===
=== [9] 4/7 (004370) ===
=== [9] 5/7 (000210) ===
=== [9] 6/7 (282330) ===
=== [9] 7/7 (111770) ===
=== [10] 1/7 (020000) ===
=== [10] 2/7 (031430) ===
=== [10] 3/7 (093050) ===
=== [10] 4/7 (004370) ===
=== [10] 5/7 (000210) ===
=== [10] 6/7 (282330) ===
=== [10] 7/7 (111770) ===
=== [11] 1/7 (020000) ===
=== [11] 2/7 (031430) ===
=== [11] 3/7 (093050) ===
=== [11] 4/7 (004370) ===
=== [11] 5/7 (000210) ===
=== [11] 6/7 (282330) ===
=== [11] 7/7 (111770) ===
=== [12] 1/7 (020000) ===
=== [12] 2/7 (031430) ===
=== [12] 3/7 (093050) ===
=== [12] 4/7 (004370) ===
=== [12] 5/7 (000210) ===
=== [12] 6/7 (282330) ===
=== [12] 7/7 (111770) ===
=== [13] 1/7 (020000) ===
=== [13] 2/7 (031430) ===
=== [13] 3/7 (093050) ===
=== [13] 4/7 (004370) ===
=== [13] 5/7 (000210) ===
=== [13] 6/7 (282330) ===
=== [13] 7/7 (111770) ===
=== [14] 1/7 (020000) ===
=== [14] 2/7 (031430) ===
=== [14] 3/7 (093050) ===
=== [14] 4/7 (00437

In [22]:
kospi_open_days[13]

'20230119'

In [27]:
len(get_kospi_open_days('20230101', '20230331'))

62

In [24]:
len(get_kospi_open_days('20230401', '20230630'))

61

In [25]:
len(get_kospi_open_days('20230701', '20230930'))

62

In [26]:
len(get_kospi_open_days('20231001', '20231231'))

60